In [2]:
#importing the required packages 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

from xgboost import XGBClassifier


In [3]:
x=pd.read_csv('x.csv')
y=pd.read_csv('y.csv')

In [4]:
x.shape

(565372, 146)

In [4]:
#splitting data into test train
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.20) # 0.25 x 0.8 = 0.2

In [5]:
#Min-max Data normalization
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scale = min_max_scaler.fit_transform(X_train)
X_val_scale = min_max_scaler.fit_transform(X_val)
X_test_scale = min_max_scaler.fit_transform(X_test)

# Modeling 

- GBM Base Model


In [6]:
from xgboost import XGBClassifier
model = XGBClassifier(learning_rate =0.02, #0.01
n_estimators=600,
max_depth=4, 
min_child_weight=4,
subsample=0.8, 
colsample_bytree=0.8,
objective= 'binary:logistic', 
nthread=6,
scale_pos_weight=5, 
seed=27)
eval_set = [(X_val_scale, y_val)]
model.fit(X_train_scale, y_train, early_stopping_rounds=3, eval_metric="logloss", eval_set=eval_set,verbose=True)

D:\conda\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\conda\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-logloss:0.68830
Will train until validation_0-logloss hasn't improved in 3 rounds.
[1]	validation_0-logloss:0.68335
[2]	validation_0-logloss:0.67881
[3]	validation_0-logloss:0.67458
[4]	validation_0-logloss:0.67018
[5]	validation_0-logloss:0.66639
[6]	validation_0-logloss:0.66284
[7]	validation_0-logloss:0.65930
[8]	validation_0-logloss:0.65556
[9]	validation_0-logloss:0.65229
[10]	validation_0-logloss:0.64946
[11]	validation_0-logloss:0.64644
[12]	validation_0-logloss:0.64281
[13]	validation_0-logloss:0.63986
[14]	validation_0-logloss:0.63708
[15]	validation_0-logloss:0.63462
[16]	validation_0-logloss:0.63144
[17]	validation_0-logloss:0.62923
[18]	validation_0-logloss:0.62721
[19]	validation_0-logloss:0.62426
[20]	validation_0-logloss:0.62178
[21]	validation_0-logloss:0.61938
[22]	validation_0-logloss:0.61747
[23]	validation_0-logloss:0.61491
[24]	validation_0-logloss:0.61263
[25]	validation_0-logloss:0.61091
[26]	validation_0-logloss:0.60795
[27]	validation_0-logloss

[236]	validation_0-logloss:0.35178
[237]	validation_0-logloss:0.35072
[238]	validation_0-logloss:0.34961
[239]	validation_0-logloss:0.34885
[240]	validation_0-logloss:0.34792
[241]	validation_0-logloss:0.34732
[242]	validation_0-logloss:0.34644
[243]	validation_0-logloss:0.34559
[244]	validation_0-logloss:0.34475
[245]	validation_0-logloss:0.34491
[246]	validation_0-logloss:0.34448
[247]	validation_0-logloss:0.34449
[248]	validation_0-logloss:0.34477
[249]	validation_0-logloss:0.34459
Stopping. Best iteration:
[246]	validation_0-logloss:0.34448



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.02, max_delta_step=0, max_depth=4,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              n_estimators=600, n_jobs=6, nthread=6, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=5, seed=27, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)

# Evaluation 

- Precision, Recall

In [8]:
#imblance classes 
y_pred = model.predict(X_test_scale)
print(classification_report(y_test, y_pred))

print('Accuracy of rf classifier on test set: {:.2f}'.format(model.score(X_test_scale, y_test)))
print("AUC of rf classifier on test set", roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.66      0.78     56438
           1       0.74      0.97      0.84     56637

    accuracy                           0.81    113075
   macro avg       0.85      0.81      0.81    113075
weighted avg       0.85      0.81      0.81    113075

Accuracy of rf classifier on test set: 0.81
AUC of rf classifier on test set 0.8140432734317168


In [15]:
from joblib import dump
from joblib import load

dump(model, "GBM_final_joblib.dat")
#print("Saved model to: credit_risk_joblib.dat")
 
# some time later...
 
## load model from file
#loaded_model = load("credit_risk_joblib.dat")

['GBM_final_joblib.dat']

# Model Tuning  

- Random Grid Search 

In [9]:
from sklearn.model_selection import cross_validate
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

clf_xgb = XGBClassifier(objective = 'binary:logistic', early_stopping_rounds=3, eval_metric="logloss", eval_set=eval_set)
param_dist = {'n_estimators': [600, 800],
              'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 1000)),
              'subsample': [0.3,0.8, 0.9],
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': list(np.linspace(0.6, 1, 10)),
              'min_child_weight': [1, 2, 3, 4,5,6],
              'scale_pos_weight': [5],
              'nthread':[6]
             }


kf = KFold(n_splits=2)
kfold_5=kf.split(x)


clf = RandomizedSearchCV(clf_xgb, param_dist,n_iter=4,verbose=1,cv=3, n_jobs = 5)


In [10]:
#X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]

#X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
search =clf.fit(X_train_scale, y_train,verbose=True)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  12 out of  12 | elapsed: 126.0min finished
D:\conda\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\conda\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#imblance classes 
y_pred = search.predict(X_test_scale)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
print('Accuracy of rf classifier on test set: {:.2f}'.format(search.score(X_test_scale, y_test)))
from sklearn.metrics import roc_auc_score
print("AUC of rf classifier on test set", roc_auc_score(y_test, y_pred))

In [5]:
#from joblib import dump
#from joblib import load

#dump(search, "credit_risk__random_serach_joblib.dat")
#print("Saved model to: credit_risk_joblib.dat")
 
# some time later...
 
## load model from file
#loaded_model = load("credit_risk_joblib.dat")